In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense
import category_encoders as ce


In [48]:
flight_df = pd.read_csv('~/Downloads/flight_data.csv')

In [49]:
flight_df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline',
       'Origin', 'Dest', 'DepTime', 'DepDelay', 'TaxiOut', 'WheelsOff',
       'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDel15',
       'Cancelled', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Carrier',
       'Full-time', 'Part-time', 'Grand Total'],
      dtype='object')

In [50]:
flight_df.describe()

,Year,Quarter,Month,DayofMonth,DayOfWeek,Flight_Number_Reporting_Airline,DepTime,DepDelay,TaxiOut,WheelsOff,...,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Full-time,Part-time,Grand Total
count,1.741288e+07,1.741288e+07,1.741288e+07,1.741288e+07,1.741288e+07,1.741288e+07,1.685512e+07,1.685488e+07,1.684936e+07,1.684936e+07,...,17412876.0,1.741288e+07,2.819049e+06,2.819049e+06,2.819049e+06,2.819049e+06,2.819049e+06,1.736074e+07,1.736074e+07,1.736074e+07
mean,2.021117e+03,2.504496e+00,6.516729e+00,1.575754e+01,3.996971e+00,2.553394e+03,1.325829e+03,8.725153e+00,1.611165e+01,1.348385e+03,...,1.0,8.073822e+02,2.671508e+01,4.121156e+00,1.153856e+01,1.721520e-01,2.348876e+01,4.207056e+04,3.975982e+03,4.604654e+04
std,8.012217e-01,1.132727e+00,3.480794e+00,8.792261e+00,2.003826e+00,1.765038e+03,4.908947e+02,4.714331e+01,8.695804e+00,4.920616e+02,...,0.0,5.826777e+02,7.441445e+01,3.271135e+01,2.921865e+01,3.915622e+00,5.412038e+01,3.302934e+04,4.485664e+03,3.623438e+04
min,2.020000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.280000e+02,1.000000e+00,1.000000e+00,...,1.0,2.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.357000e+03,0.000000e+00,2.374000e+03
25%,2.020000e+03,1.000000e+00,3.000000e+00,8.000000e+00,2.000000e+00,1.065000e+03,9.190000e+02,-6.000000e+00,1.100000e+01,9.340000e+02,...,1.0,3.830000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.099600e+04,9.980000e+02,1.319100e+04
50%,2.021000e+03,3.000000e+00,7.000000e+00,1.600000e+01,4.000000e+00,2.189000e+03,1.320000e+03,-3.000000e+00,1.400000e+01,1.332000e+03,...,1.0,6.600000e+02,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.336700e+04,1.805000e+03,5.481200e+04
75%,2.022000e+03,4.000000e+00,1.000000e+01,2.300000e+01,6.000000e+00,3.914000e+03,1.731000e+03,5.000000e+00,1.900000e+01,1.744000e+03,...,1.0,1.045000e+03,2.600000e+01,0.000000e+00,1.600000e+01,0.000000e+00,2.600000e+01,6.912400e+04,5.655000e+03,7.880900e+04
max,2.022000e+03,4.000000e+00,1.200000e+01,3.100000e+01,7.000000e+00,9.888000e+03,2.400000e+03,3.890000e+03,2.560000e+02,2.400000e+03,...,1.0,5.812000e+03,3.864000e+03,2.363000e+03,1.740000e+03,1.245000e+03,2.361000e+03,9.737300e+04,1.642400e+04,1.091080e+05


## Shared Preprocessing

In [51]:
flight_df = flight_df[flight_df.Cancelled == 0]
flight_df = flight_df[flight_df.Diverted == 0]
len(flight_df)

16809809

In [52]:
# Reducing dimensionality of Carrier with custom grouping
value_counts = flight_df['Carrier'].value_counts()
to_remove = value_counts[value_counts <= 9000].index
flight_df['Carrier'].replace(to_remove, 'Other', inplace=True)

In [53]:
# Reducing dimensionality of TimeZones 
def binary_encode(df, columns):
    encoder = ce.BinaryEncoder(cols=columns)

    # Fit and transform to produce binary encoded data
    df_encoded = encoder.fit_transform(df[columns])

    # Merge the encoded data back with the original DataFrame
    df = df.drop(columns, axis=1)
    df = pd.concat([df, df_encoded], axis=1)
    return df

In [54]:
def label_encode_airport_codes(df):
    label_encoder = LabelEncoder()
    label_encoder.fit(df['Origin'].append(df['Dest']).unique())

    # Convert Origin and Dest to integer indices
    df['Origin_encoded'] = label_encoder.transform(df['Origin'])
    df['Dest_encoded'] = label_encoder.transform(df['Dest'])

    df = df.drop(['Dest', 'Origin'], axis=1)
    return df

## Model run 1

Timezones instead of airports

In [55]:
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
        'Dest', 'Origin', 'Distance', 'Carrier',
        'Full-time', 'Part-time', 'Grand Total', 'CRSArrTime', 'CRSElapsedTime']
len(features)

14

In [56]:
temp_features = features + ['ArrDel15']
flight_df = flight_df[temp_features]
flight_df = flight_df.dropna()

y = flight_df['ArrDel15']
X = flight_df[features]

In [57]:
# Encoding categorical variables
X = binary_encode(X, ['Dest', 'Origin'])
X = pd.get_dummies(X, columns=['Carrier'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [58]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
335237/335237 [==============================] - 155s 462us/step - loss: 1.4673 - accuracy: 0.8255 - val_loss: 0.4525 - val_accuracy: 0.8321
Epoch 2/10
335237/335237 [==============================] - 157s 468us/step - loss: 0.4527 - accuracy: 0.8322 - val_loss: 0.4525 - val_accuracy: 0.8321
Epoch 3/10
335237/335237 [==============================] - 158s 473us/step - loss: 0.4527 - accuracy: 0.8322 - val_loss: 0.4526 - val_accuracy: 0.8321
Epoch 4/10
335237/335237 [==============================] - 157s 469us/step - loss: 0.4527 - accuracy: 0.8322 - val_loss: 0.4526 - val_accuracy: 0.8321
Epoch 5/10
335237/335237 [==============================] - 155s 463us/step - loss: 0.4527 - accuracy: 0.8322 - val_loss: 0.4525 - val_accuracy: 0.8321
Epoch 6/10
 28137/335237 [=>............................] - ETA: 2:07 - loss: 0.4520 - accuracy: 0.8324

KeyboardInterrupt: 

In [59]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
        'Dest', 'Origin', 'Distance', 'Carrier',
        'Full-time', 'Part-time', 'Grand Total', 'CRSArrTime', 'CRSElapsedTime']
        
# Test accuracy: 0.8320582509040833
# Test loss: 0.45254066586494446

104762/104762 [==============================] - 27s 262us/step - loss: 0.4525 - accuracy: 0.8321
Test accuracy: 0.8320582509040833
Test loss: 0.45254066586494446


In [46]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)


features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
       'Dest', 'Origin',
       'DepTime', 'DepDelay','TaxiOut', 'WheelsOff', 
       'Distance', 'Carrier','Full-time', 'Part-time', 'Grand Total', 'CRSArrTime']
# Test accuracy: 0.949720561504364
# Test loss: 0.1481556445360183

104762/104762 [==============================] - 25s 236us/step - loss: 0.1482 - accuracy: 0.9497
Test accuracy: 0.949720561504364
Test loss: 0.14815564453601837
